# Example of computing vibrational energy levels of a triatomic molecule using the Taylor series expansion approximation for the kinetic and potential energy operators

Compute vibrational states of $\text{H}_2\text{S}$ molecule using the potential energy surface from
[A. A. A. Azzam, J. Tennyson, S. N. Yurchenko, O. V. Naumenko, "ExoMol molecular line lists - XVI. The rotation–vibration spectrum of hot H2S", MNRAS 460, 4063–4074 (2016)](https://doi.org/10.1093/mnras/stw1133)


In [1]:
import itertools
import os

import jax
import jax.numpy as jnp
import numpy as np
from scipy import optimize

from vibrojet.basis_utils import ContrBasis, HermiteBasis
from vibrojet.keo import Gmat, pseudo, com
from vibrojet.potentials import h2s_AYT2
from vibrojet.taylor import deriv_list

jax.config.update("jax_enable_x64", True)

Compute equilibrium coordinates, around which the Taylor series expansions will be built

In [2]:
vmin = optimize.minimize(h2s_AYT2.poten, [1, 1, 90 * np.pi / 180])
r0 = vmin.x
v0 = vmin.fun
print("Equilibrium coordinates:", r0)
print("Min of the potential:", v0)

Equilibrium coordinates: [1.3358387  1.3358387  1.61042427]
Min of the potential: -0.0007846164047977397


Define mapping function from internal valence coordinates, $r_1$, $r_2$, and $\alpha$, to Cartesian coordinates

In [3]:
masses = [31.97207070, 1.00782505, 1.00782505]  # masses of S, H, H

ncoo = len(r0)


@com(masses)
def internal_to_cartesian(coords):
    r1, r2, alpha = coords
    return jnp.array(
        [
            [0.0, 0.0, 0.0],
            [r1 * jnp.cos(alpha / 2), 0.0, r1 * jnp.sin(alpha / 2)],
            [r2 * jnp.cos(alpha / 2), 0.0, -r2 * jnp.sin(alpha / 2)],
        ]
    )

Compute Taylor series expansion of the $G$-matrix and potential

$$
G_{\lambda,\mu}=\sum_{\mathbf{t}} g_{\mathbf{t},\lambda,\mu} (r_1-r_1^\text{(eq)})^{t_1}(r_2-r_2^\text{(eq)})^{t_2}(\alpha-\alpha^\text{(eq)})^{t_3},
$$

$$
V=\sum_{\mathbf{t}} f_{\mathbf{t}} (r_1-r_1^\text{(eq)})^{t_1}(r_2-r_2^\text{(eq)})^{t_2}(\alpha-\alpha^\text{(eq)})^{t_3},
$$

 where the derivative multi-index $\mathbf{t}$ is stored in `gmat_terms` and `poten_terms`, and the expansion coefficients $g_{\mathbf{t},\lambda,\mu}$ and $f_{\mathbf{t}}$ in `gmat_coefs[:len(gmat_terms),:3N, :3N]` and `poten_coefs[:len(poten_terms)]` respectively, where $N$ is the number of atoms


In [4]:
gmat_max_order = 8  # max total expansion order

gmat_terms = np.array(
    [
        elem
        for elem in itertools.product(
            *[range(0, gmat_max_order + 1) for _ in range(len(r0))]
        )
        if sum(elem) <= gmat_max_order
    ]
)
pseudo_terms = gmat_terms
print("max expansion order for G-matrix:", gmat_max_order)
print("number of expansion terms in G-matrix:", len(gmat_terms))

# expansion of G-matrix

gmat_file = f"_h2s_gmat_coefs_{gmat_max_order}.npy"
if os.path.exists(gmat_file):
    print(
        f"load G-matrix expansion coefs from file {gmat_file} (delete file to recompute coefficients)"
    )
    gmat_coefs = np.load(gmat_file)
else:
    gmat_coefs = deriv_list(
        lambda x: Gmat(x, masses, internal_to_cartesian), gmat_terms, r0, if_taylor=True
    )
    np.save(gmat_file, gmat_coefs)

# expansion of pseudopotential

pseudo_file = f"_h2s_pseudo_coefs_{gmat_max_order}.npy"
if os.path.exists(pseudo_file):
    print(
        f"load pseudopotential expansion coefs from file {pseudo_file} (delete file to recompute coefficients)"
    )
    pseudo_coefs = np.load(pseudo_file)
else:
    pseudo_coefs = deriv_list(
        lambda x: pseudo(x, masses, internal_to_cartesian),
        gmat_terms,
        r0,
        if_taylor=True,
    )
    np.save(pseudo_file, pseudo_coefs)

max expansion order for G-matrix: 8
number of expansion terms in G-matrix: 165
load G-matrix expansion coefs from file _h2s_gmat_coefs_8.npy (delete file to recompute coefficients)
load pseudopotential expansion coefs from file _h2s_pseudo_coefs_8.npy (delete file to recompute coefficients)


In [5]:
poten_max_order = 8

poten_terms = np.array(
    [
        elem
        for elem in itertools.product(
            *[range(0, poten_max_order + 1) for _ in range(len(r0))]
        )
        if sum(elem) <= poten_max_order
    ]
)
print("max expansion order for PES:", poten_max_order)
print("number of expansion terms in PES:", len(poten_terms))

poten_file = f"_h2s_poten_coefs_{poten_max_order}.npy"
if os.path.exists(poten_file):
    print(
        f"load potential expansion coefs from file {poten_file} (delete file to recompute coefficients)"
    )
    poten_coefs = np.load(poten_file)
else:
    poten_coefs = deriv_list(h2s_AYT2.poten, poten_terms, r0, if_taylor=True)
    np.save(poten_file, poten_coefs)

max expansion order for PES: 8
number of expansion terms in PES: 165
load potential expansion coefs from file _h2s_poten_coefs_8.npy (delete file to recompute coefficients)


Define mapping between the coordinate $x\in (-\infty, \infty)$ of Hermite basis functions and the internal valence coordinates as $r_1=a_1 x_1 + b_1$, $r_2=a_2x_2+b_2$, $\alpha=a_3x_3+b_3$. The parameters $a_1,b_1,...,b_3$ are determined by mapping the vibrational Hamiltonian in valence coordinates onto the harmonic-oscillator Hamiltonian.

In [6]:
mask = gmat_terms != 0  # Mask for derivatives

# Vibrational G-matrix elements at equilibrium geometry
ind0 = np.where(mask.sum(axis=1) == 0)[0][0]
mu = np.diag(gmat_coefs[ind0])[:ncoo]

# Second-order derivative of potential at equilibrium
mask = poten_terms != 0  # Mask for derivatives
ind2 = np.array(
    [   np.where((mask.sum(axis=1) == 1) & (poten_terms[:, icoo] == 2))[0][0]
        for icoo in range(ncoo)]
)

freq = poten_coefs[ind2] * 2

# Linear mapping parameters
lin_a = jnp.sqrt(jnp.sqrt(mu / freq))
lin_b = r0

print("x->r linear mapping parameters 'a':", lin_a)
print("x->r linear mapping parameters 'b':", lin_b)

# Linear mapping function and its derivative
x_to_r_map = lambda x, icoo: lin_a[icoo] * x + lin_b[icoo]

x->r linear mapping parameters 'a': [0.11245619 0.11245619 0.17845517]
x->r linear mapping parameters 'b': [1.3358387  1.3358387  1.61042427]


In [7]:
nbas = [30, 30, 24]
npoints = [80] * ncoo

bas_r1, bas_r2, bas_alpha = [
    HermiteBasis(
        icoo,
        nbas[icoo],
        npoints[icoo],
        lambda x: x_to_r_map(x, icoo),
        lambda r: r - r0[icoo],
        lambda r: r - r0[icoo],
        gmat_terms[:, icoo],
        poten_terms[:, icoo],
        pseudo_terms[:, icoo],
    )
    for icoo in range(ncoo)
]

In [8]:
bas_r1, bas_r2, bas_alpha = [
    ContrBasis(
        (icoo,),
        [bas_r1, bas_r2, bas_alpha],
        lambda _: True,
        gmat_terms,
        gmat_coefs,
        poten_terms,
        poten_coefs,
        emax=40000,
    )
    for icoo in range(ncoo)
]

e = bas_r1.enr
print("Solutions for r1 mode:\n", f"zpe = {e[0]}\n", e - e[0])
e = bas_r2.enr
print("\nSolutions for r2 mode:\n", f"zpe = {e[0]}\n", e - e[0])
e = bas_alpha.enr
print("\nSolutions for alpha mode:\n", f"zpe = {e[0]}\n", e - e[0])

Solutions for r1 mode:
 zpe = 2361.7950570907624
 [    0.          2627.83310095  5165.68583648  7629.77901979
 10056.05642783 12496.83825129 15001.60164644 17601.69818093
 20311.16466355 23134.07657558 26069.98236833 29116.65213762
 32271.47762591 35532.58893808]

Solutions for r2 mode:
 zpe = 2361.795057137522
 [    0.          2627.83310095  5165.68583648  7629.77901976
 10056.05642772 12496.83825106 15001.60164602 17601.69818028
 20311.16466265 23134.07657442 26069.98236687 29116.65213587
 32271.47762385 35532.5889357 ]

Solutions for alpha mode:
 zpe = 2026.5007139178238
 [    0.          1214.1828581   2422.52098472  3624.57606553
  4819.78084361  6007.22739247  7185.15140905  7351.13931048
  8350.72917815  9496.80946149 10608.78971238 11656.6267001
 12623.30073129 13598.36265685 14676.14362905 15845.09195459
 17086.46640086 18383.87068341 19733.52931523 21161.16514744
 22547.85661979 24182.30837895 25650.8574639  27516.3243721 ]


In [9]:
def select_quanta(ind):
    cond =np.sum(ind)<=10
    # if cond:
    #     print(ind)
    return cond

bas_r1_r2 = ContrBasis(
    (0, 1),
    (bas_r1, bas_r2, bas_alpha),
    select_quanta,
    gmat_terms,
    gmat_coefs,
    poten_terms,
    poten_coefs,
    emax=40000,
)

e = bas_r1_r2.enr
print("Solutions for r1+r2 modes:\n", f"zpe = {e[0]}\n", e - e[0])

Solutions for r1+r2 modes:
 zpe = 3007.6990842319456
 [    0.          2619.90705667  2634.8276465   5162.02005279
  5164.09721121  5256.79160138  7624.76506304  7624.98092962
  7777.25184655  7808.28535541 10043.99499206 10044.16583823
 10250.03809698 10254.24460664 10341.27565912 12465.94443921
 12465.9880656  12675.59150262 12680.95873993 12784.76619573
 12831.35654899 14922.64250302 14922.81774822 15113.73208083
 15120.85597332 15236.04565629 15244.47015989 15316.94989962
 17415.62402997 17416.78502129 17591.76911473 17605.69269665
 17708.08285624 17712.16906552 17783.01057169 17827.16046844
 19943.09630031 19948.31598366 20086.09714161 20110.66965888
 20211.94617504 20218.17776495 20287.50073459 20451.59139603
 20461.01277018 22509.70899216 22530.21894059 22594.90067929
 22635.29051442 22738.85409258 22796.31685408 22909.43217483
 22953.78071497 23271.99211995 23274.18600127 25078.04379493
 25106.78947529 25205.36068661 25229.12038641 25384.35159444
 25464.2595517  25573.17196892 

In [10]:
p_max = 4


def select_quanta(ind):
    ind_str, ind_bnd = ind
    cond = int(np.ceil(ind_str / 2)) * 2 + ind_bnd <= p_max
    # cond = ind_str + ind_bnd <= p_max
    # if cond:
    #     print(ind)
    return cond


bas_r1_r2_alpha = ContrBasis(
    (0, 1),
    (bas_r1_r2, bas_alpha),
    select_quanta,
    gmat_terms,
    gmat_coefs,
    poten_terms,
    poten_coefs,
)

e = bas_r1_r2_alpha.enr
print(len(e))
print("Solutions for r1+r2+alpha modes:\n", f"zpe = {e[0]}\n", e - e[0])

13
Solutions for r1+r2+alpha modes:
 zpe = 3300.8614510239067
 [   0.         1182.77903903 2361.24957436 2619.11753962 2631.05710132
 3549.27328806 3797.31855916 3798.73203777 4721.75186548 4966.84620744
 4976.62818808 5158.44031537 5159.68186747]


In [11]:
p_coefs = np.array([2, 2, 1])
p_max = 4


def select_quanta(ind):
    cond = np.sum(np.array(ind) * p_coefs) <= p_max
    # if cond:
    #     print(ind)
    return cond


bas_r1_r2_alpha = ContrBasis(
    (0, 1, 2),
    (bas_r1, bas_r2, bas_alpha),
    select_quanta,
    gmat_terms,
    gmat_coefs,
    poten_terms,
    poten_coefs,
)

e = bas_r1_r2_alpha.enr
print(len(e))
print("Solutions for r1+r2+alpha modes:\n", f"zpe = {e[0]}\n", e - e[0])

14
Solutions for r1+r2+alpha modes:
 zpe = 3300.860811017275
 [   0.         1182.79323452 2361.27810455 2618.96029563 2631.08481639
 3549.45945933 3797.31861866 3798.92604393 4721.98390385 4967.14211047
 4976.73952383 5158.62053948 5159.9018466  5252.05117218]
